In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#Importing Libraries
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [ ]:
import os 
#Loading the list of chopin's midi files as stream 
filepath = "/Users/taeyoon/Downloads/archive/chopin/"
#Getting midi files
all_midis= []
for i in os.listdir(filepath):
    print(i)
    if i.endswith(".mid"):
        tr = filepath+i
        try:
            if os.path.exists(filepath):
                #print("file path exists!")
                midi = converter.parse(tr)
                all_midis.append(midi)
            else: 
                print("file does not exist")
        except Exception as e: 
            #pass
            print(f"An error occurred: {e}")
all_midis

In [ ]:
#Helping function        
def extract_notes(file):
    notes = []
    duration = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                    duration.append(element.quarterLength)
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))
                    #chord_notes = '.'.join(str(n) for n in element.pitches)
                    durations = element.duration.quarterLength
                    duration.append(durations)

    return notes, duration
#Getting the list of notes as Corpus
Corpus, NoteDurations = extract_notes(all_midis)
print("Total notes in all the Chopin midis in the dataset:", len(Corpus))

In [ ]:
print("First fifty values in the Corpus:", Corpus[:50])

In [ ]:
print("First fifty nute durations in the Corpus:", NoteDurations[:50])

In [ ]:
#Creating a count dictionary
count_num = Counter(Corpus)
print("Total unique notes in the Corpus:", len(count_num))

In [ ]:
#Creating a count dictionary
dur_count_num = Counter(NoteDurations)
print("Total unique notes in the Corpus:", len(dur_count_num)) 

In [ ]:
#Exploring the notes dictionary
Notes = list(count_num.keys())
Recurrence = list(count_num.values())
#Average recurrenc for a note in Corpus
def Average(lst):
    return sum(lst) / len(lst)
print("Average recurrenc for a note in Corpus:", Average(Recurrence))
print("Most frequent note in Corpus appeared:", max(Recurrence), "times")
print("Least frequent note in Corpus appeared:", min(Recurrence), "time")

In [ ]:
# Plotting the distribution of Notes
plt.figure(figsize=(18,3),facecolor="#97BACB")
bins = np.arange(0,(max(Recurrence)), 50) 
plt.hist(Recurrence, bins=bins, color="#97BACB")
plt.axvline(x=100,color="#DBACC1")
plt.title("Frequency Distribution Of Notes In The Corpus")
plt.xlabel("Frequency Of Chords in Corpus")
plt.ylabel("Number Of Chords")
plt.show()

In [ ]:
# Getting a list of rare chords
rare_note = [key for key, value in count_num.items() if value < 100]

print("Total number of notes that occur less than 100 times:", len(rare_note))

# Eliminating the rare notes and corresponding durations
filtered_Corpus = []
filtered_Durations = []

for notee, duration in zip(Corpus, NoteDurations):
    if notee not in rare_note:
        filtered_Corpus.append(notee)
        filtered_Durations.append(duration)
        
print("Length of Corpus after eliminating the rare notes:", len(filtered_Corpus))
print("Length of NotesDurations after eliminating the rare notes:", len(filtered_Durations))



In [ ]:
# Updating the original lists
Corpus = filtered_Corpus
NotesDurations = filtered_Durations

# Printing the final lists (optional)
print("Filtered Corpus:", Corpus)
print("Filtered NotesDurations:", NotesDurations)

In [ ]:
# Storing all the unique characters present in my corpus to bult a mapping dict. 
symb = sorted(list(set(Corpus)))

L_corpus = len(Corpus) #length of corpus
L_symb = len(symb) #length of total unique characters
L_symb_dur = len(list(set(NoteDurations))) #length of total unique note durations

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)

In [ ]:
# Storing all the unique characters present in my corpus to bult a mapping dict. 
symb_2 = sorted(list(set(NoteDurations)))
symb_2_durations = len(NoteDurations)


#Building dictionary to access the vocabulary from indices and vice versa
mapping_2 = dict((c, i) for i, c in enumerate(symb_2))
reverse_mapping_2= dict((i, c) for i, c in enumerate(symb_2))

print("Total number of durations:", symb_2_durations)
print("Number of unique durations:", L_symb_dur)

In [ ]:
#Splitting the Corpus in equal length of strings and output target
length = 40
note_features = []
note_targets = []
duration_features = []
duration_targets = []
for i in range(0, L_corpus - length, 1):
    note_feature = Corpus[i:i + length]
    note_target = Corpus[i + length]
    
    duration_feature = NoteDurations[i:i + length]
    duration_target = NoteDurations[i + length]
    
    note_features.append([mapping[j] for j in note_feature])
    note_targets.append(mapping[note_target])
    
    duration_features.append([mapping_2[j] for j in duration_feature])
    duration_targets.append(mapping_2[duration_target])
    
    
    
L_datapoints = len(note_targets)
print("Total number of sequences in the Corpus:", L_datapoints)

In [ ]:
# reshape X and normalize
X_notes = (np.reshape(note_features, (L_datapoints, length, 1)))/ float(L_symb)
X_durations = (np.reshape(duration_features, (L_datapoints, length, 1)))/ float(L_symb_dur)
# one hot encode the output variable
y_notes = tensorflow.keras.utils.to_categorical(note_targets) 
y_durations = tensorflow.keras.utils.to_categorical(duration_targets) 
y = np.concatenate([y_notes, y_durations], axis=1)

In [ ]:
X_durations = X_durations.astype(np.float64) #crucial

In [ ]:
#Taking out a subset of data to be used as seed
train_idx, test_idx = train_test_split(np.arange(np.shape(y_notes)[0]), random_state=42)
X_train_notes, X_train_durations, y_train_notes, y_train_durations = X_notes[train_idx], X_durations[train_idx], y_notes[train_idx], y_durations[train_idx]
X_seed_notes, X_seed_durations, y_seed_notes, y_seed_durations = X_notes[test_idx], X_durations[test_idx], y_notes[test_idx], y_durations[test_idx]
#X_train, X_seed, y_train, y_seed = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.layers import Input, LSTM, Dropout, Concatenate, Dense
from keras.models import Model
from keras.optimizers import Adamax

# Input branches
notes_input = Input(shape=(X_train_notes.shape[1], X_train_notes.shape[2]), name='notes_input')
durations_input = Input(shape=(X_train_durations.shape[1], X_train_durations.shape[2]), name='durations_input')

# Notes branch
notes_lstm1 = LSTM(512, return_sequences=True)(notes_input)
notes_dropout1 = Dropout(0.1)(notes_lstm1)
notes_lstm2 = LSTM(256)(notes_dropout1)

# Durations branch
durations_lstm1 = LSTM(512, return_sequences=True)(durations_input)
durations_dropout1 = Dropout(0.1)(durations_lstm1)
durations_lstm2 = LSTM(256)(durations_dropout1)

# Concatenation of both branches
concatenated = Concatenate()([notes_lstm2, durations_lstm2])

# Dense layers with dropout
dense1 = Dense(256)(concatenated)
dropout2 = Dropout(0.1)(dense1)

# Output layers for notes and durations
notes_output = Dense(y_notes.shape[1], activation='softmax', name='notes_output')(dropout2)
durations_output = Dense(y_durations.shape[1], activation='softmax', name='durations_output')(dropout2)

# Define and compile the model
model = Model(inputs=[notes_input, durations_input], outputs=[notes_output, durations_output])
opt = Adamax(learning_rate=0.01)
model.compile(loss={'notes_output': 'categorical_crossentropy', 'durations_output': 'categorical_crossentropy'},
              optimizer=opt,
              metrics={'notes_output': 'accuracy', 'durations_output': 'accuracy'})

# Summary of the model
model.summary()


In [ ]:
def chords_n_notes(Snippet, dur_lst):  
    Melody = []  
    offset = 0 #Incremental  
    for duration, i in zip(dur_lst, Snippet):  
        #If it is chord  
        if ("." in i or i.isdigit()):  
            chord_notes = i.split(".") #Seperating the notes in chord  
            notes = []   
            for j in chord_notes:  
                inst_note=int(j)  
                note_snip = note.Note(inst_note)              
                notes.append(note_snip)  
                chord_snip = chord.Chord(notes)  
                chord_snip.offset = offset  
                chord_snip.quarterLength = duration  
                Melody.append(chord_snip)  
        # pattern is a note  
        else:   
            note_snip = note.Note(i) #why is note = 2.7 here?   
            note_snip.offset = offset  
            note_snip.duration.quarterLength = duration  
              
            Melody.append(note_snip)  
        # increase offset each iteration so that notes do not stack  
        offset += duration  
    Melody_midi = stream.Stream(Melody)     
    return Melody_midi  


In [ ]:
y_train_durations

In [ ]:
y_train_notes

In [ ]:
history = model.fit([X_train_notes, X_train_durations], [y_train_notes, y_train_durations], epochs=50, batch_size=64)

In [ ]:
#Plotting the learnings 
history_df = pd.DataFrame(history.history)
fig = plt.figure(figsize=(15,4), facecolor="#97BACB")
fig.suptitle("Learning Plot of Model for Loss")
pl=sns.lineplot(data=history_df["loss"],color="#444160")
pl.set(ylabel ="Training Loss")
pl.set(xlabel ="Epochs")

**Generating the Melody**

In [ ]:


def preprocess_prediction(prediction): 
    L_symb_tmp = float(prediction.shape[-1])
    prediction = np.log(prediction) / 1.0 #diversity
    exp_preds = np.exp(prediction)
    prediction = exp_preds / np.sum(exp_preds)
    index = np.argmax(prediction)
    index_N = index/ float(L_symb_tmp)   
    
    return index, index_N

def Malody_Generator(Note_Count):
    notes_seed = X_seed_notes[np.random.randint(0,len(X_seed_notes)-1)]
    durations_seed = X_seed_durations[np.random.randint(0,len(X_seed_durations)-1)]
    Music = ""
    Notes_Generated=[]
    Durations_Generated=[]
    for i in range(Note_Count):
        notes_seed = notes_seed.reshape(1,length,1)
        durations_seed = durations_seed.reshape(1,length,1)
        
        note_prediction, duration_prediction = model.predict([notes_seed, durations_seed], verbose=0)
        note_index, note_index_N = preprocess_prediction(note_prediction)
        duration_index, duration_index_N = preprocess_prediction(duration_prediction)
        
        Notes_Generated.append(note_index)
        
        Music = [reverse_mapping[char] for char in Notes_Generated]
        notes_seed = np.insert(notes_seed[0],len(notes_seed[0]),note_index_N) #What is note_index_N doing for us? Something, not sure what tho. Need to double check this. 
        notes_seed = notes_seed[1:]
        
        Durations_Generated.append(duration_index)

        Music_2 = [reverse_mapping_2[char] for char in Durations_Generated]
        durations_seed = np.insert(durations_seed[0],len(durations_seed[0]),duration_index_N)
        durations_seed = durations_seed[1:]
    #Now, we have music in form or a list of chords and notes and we want to be a midi file.
    Melody = chords_n_notes(Music, Music_2) #apparently, the data inside of Music should be of type Note.note()? 
    Melody_midi = stream.Stream(Melody)   
    return Music,Melody_midi


#getting the Notes and Melody created by the model
Music_notes, Melody = Malody_Generator(100)



In [ ]:
#To save the generated melody
Melody.write('midi','Melody_Generated.mid')


